### Title: Data preparation
### Author: Agnes Piecyk
### Content:
#### (1) imports the files umsatzdaten_gekuerzt.csv, kiwo.csv and wetter.csv from an URL and stores the merged data in a pandas dataframe
#### (2) includes a brief data exploration 
#### (3) removes rows with NaN values (n=2309)
#### (4) converts "Datum" into datetime and adds new columns for "Wochentag", "Monat" and "Jahr"

In [1]:
# import library
import pandas as pd # library for data manipulation

# import data from csv files
url1 = "https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/kiwo.csv"
url2 = "https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/umsatzdaten_gekuerzt.csv"
url3 = "https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/wetter.csv"

# Read the CSVs
kiwo = pd.read_csv(url1)
umsatz = pd.read_csv(url2)
wetter = pd.read_csv(url3)

# Merge the dataframes in two steps
## Merge umsatz and wetter dataframes
df = pd.merge(umsatz, wetter, on="Datum", how="inner")
## Merge with kiwo dataframe
df = pd.merge(df, kiwo, on=['Datum'], how='left')
df['KielerWoche'] = df['KielerWoche'].fillna(0).astype(int) # fill missing values with 0 and convert to integer

# export the merged dataframe to a csv file
df.to_csv("merged_data.csv", index=False)

In [ ]:
# Brief data exploration
print(df.head()) #  print the first 5 rows of the dataframe
print(df.info()) # print information about the dataframe (9318 entries)
print(df.isnull().sum()) # check for missing values
print(df.shape) # print the number of rows and columns in the dataframe

       Datum  Warengruppe      Umsatz  Bewoelkung  Temperatur  \
0 2013-07-01            1  148.828353         6.0     17.8375   
2 2013-07-03            1  111.885594         7.0     21.0750   
3 2013-07-04            1  168.864941         7.0     18.8500   
7 2013-07-08            1  135.500244         0.0     22.7250   
9 2013-07-10            1  135.132314         2.0     19.6875   

   Windgeschwindigkeit  Wettercode  KielerWoche  Wochentag Monat  Jahr  
0                   15        20.0            0     Monday  July  2013  
2                    6        61.0            0  Wednesday  July  2013  
3                    7        20.0            0   Thursday  July  2013  
7                   10         5.0            0     Monday  July  2013  
9                   13         5.0            0  Wednesday  July  2013  
<class 'pandas.core.frame.DataFrame'>
Index: 7009 entries, 0 to 9317
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------ 

In [10]:
# check wether there are duplicates
print(df.duplicated().sum())

0


In [11]:
# Get descriptive statistics, e.g. to check for outliers
print(df.describe())

                               Datum  Warengruppe       Umsatz   Bewoelkung  \
count                           7009  7009.000000  7009.000000  7009.000000   
mean   2016-03-11 22:43:34.352974592     3.100014   200.362704     5.307319   
min              2013-07-01 00:00:00     1.000000     7.051201     0.000000   
25%              2014-12-18 00:00:00     2.000000    94.681895     4.000000   
50%              2016-03-16 00:00:00     3.000000   155.268163     6.000000   
75%              2017-06-19 00:00:00     4.000000   271.400924     7.000000   
max              2018-07-30 00:00:00     6.000000  1879.461831     8.000000   
std                              NaN     1.497091   140.506394     2.394701   

        Temperatur  Windgeschwindigkeit   Wettercode  KielerWoche         Jahr  
count  7009.000000          7009.000000  7009.000000  7009.000000  7009.000000  
mean     10.726434            11.174775    36.900128     0.023541  2015.689542  
min      -8.475000             3.000000     0

In [4]:
# simplify the dataframe by removing rows with NaN values
## remove rows with NaN values
df = df.dropna()

## check the df without NaN values
print(df.info()) ### 7009 entries

## export the merged dataframe without NaN values to a csv file
df.to_csv("merged_data_withoutNaN.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 7009 entries, 0 to 9317
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Datum                7009 non-null   object 
 1   Warengruppe          7009 non-null   int64  
 2   Umsatz               7009 non-null   float64
 3   Bewoelkung           7009 non-null   float64
 4   Temperatur           7009 non-null   float64
 5   Windgeschwindigkeit  7009 non-null   int64  
 6   Wettercode           7009 non-null   float64
 7   KielerWoche          7009 non-null   int64  
dtypes: float64(4), int64(3), object(1)
memory usage: 492.8+ KB
None
